In [1]:
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
dataset = load_dataset("paws","labeled_final")
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


/home/divyansh/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/divyansh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
with open('GloVe/vectors.txt', 'r') as f:
    vectors=f.readlines()

In [3]:
def remove_stopwords(sentence):
    stop_words = set(stopwords.words('english'))
    words = sentence.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

def get_avg(sentence):

    words = sentence.split()
    avg = [0]*50
    for word in words:
        for vector in vectors:
            if word in vector:
                vector = vector.split()
                for i in range(50):
                    avg[i] += float(vector[i+1])
                break
    for i in range(50):
        avg[i] /= len(words)
    return avg



In [4]:
training_data = dataset['train']
testing_data = dataset['test']
validation_data = dataset['validation']

training_data = training_data.map(lambda x: {'sentence1': remove_stopwords(x['sentence1']), 'sentence2': remove_stopwords(x['sentence2'])})
testing_data = testing_data.map(lambda x: {'sentence1': remove_stopwords(x['sentence1']), 'sentence2': remove_stopwords(x['sentence2'])})
validation_data = validation_data.map(lambda x: {'sentence1': remove_stopwords(x['sentence1']), 'sentence2': remove_stopwords(x['sentence2'])})

training_vec = []


for i in range(len(training_data)):
    training_vec.append(cosine_similarity(get_avg(training_data[i]['sentence1']),get_avg(training_data[i]['sentence2'])))
y = np.array(training_data['label'])
X = np.array(training_vec)

X = (X-np.mean(X, axis=0))/np.std(X, axis=0)

mean = np.mean(X, axis=0)
std = np.std(X, axis=0)

model = LogisticRegression()

   



Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map: 100%|██████████| 8000/8000 [00:00<00:00, 9719.41 examples/s]
/tmp/ipykernel_22700/2647009791.py:8: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


KeyboardInterrupt: 

In [ ]:
model.fit(X, y)

testing_vec = []

for i in range(len(testing_data)):
    testing_vec.append(cosine_similarity(get_avg(testing_data[i]['sentence1']),get_avg(testing_data[i]['sentence2'])))

y_test = np.array(testing_data['label'])
X_test = np.array(testing_vec)
X_test = (X_test-mean)/std

y_pred = model.predict(X_test)

print(accuracy_score(y_test, y_pred)) 